In [1]:
from attack import Attack
from dataclasses import dataclass, field
from pyvis.network import Network

In [2]:
a = Attack()

In [48]:
net = Network(
    height='750px',
    width='1200px',
    notebook=True
)

In [49]:
for tac in a.cfg.tactics_order:
    tactic = tac.replace('-', ' ').title()
    net.add_node(tac, label=tactic, group=tac, color='#000')

In [50]:
for tech in a.techniques:
    
    if '.' in tech:
        parent = tech.split('.')[0]
        if parent not in net.node_ids:
            net.add_node(parent, label=a.techniques[parent].name, color="#7A1E16")
            net.add_edge(a.techniques[parent].tactics[0], parent)
        net.add_node(tech, label=a.techniques[tech].name, color="#FA3F2D")
        net.add_edge(parent, tech)
    else:
        if tech not in net.node_ids:
            net.add_node(tech, label=a.techniques[tech].name, color="#7A1E16")
            net.add_edge(a.techniques[tech].tactics[0], tech)
        
net.show_buttons()

In [3]:
a.techniques_by_mitigation

{'M1036': {'T1110', 'T1110.001', 'T1110.003', 'T1110.004'},
 'M1015': {'T1003',
  'T1003.005',
  'T1003.006',
  'T1072',
  'T1081',
  'T1097',
  'T1134.005',
  'T1178',
  'T1550.003',
  'T1552',
  'T1552.006',
  'T1558',
  'T1558.001',
  'T1606.002'},
 'M1049': {'T1027',
  'T1027.002',
  'T1045',
  'T1059',
  'T1059.001',
  'T1059.005',
  'T1059.006',
  'T1193',
  'T1194',
  'T1215',
  'T1221',
  'T1547.006',
  'T1566',
  'T1566.001',
  'T1566.003'},
 'M1013': {'T1078', 'T1547.011', 'T1564.009', 'T1574', 'T1574.002'},
 'M1048': {'T1021.003',
  'T1064',
  'T1068',
  'T1173',
  'T1175',
  'T1189',
  'T1190',
  'T1203',
  'T1210',
  'T1211',
  'T1212',
  'T1559',
  'T1559.001',
  'T1559.002',
  'T1611'},
 'M1047': {'T1021.001',
  'T1021.005',
  'T1031',
  'T1034',
  'T1038',
  'T1044',
  'T1053',
  'T1053.001',
  'T1053.002',
  'T1053.003',
  'T1053.004',
  'T1053.005',
  'T1059.006',
  'T1073',
  'T1076',
  'T1078',
  'T1081',
  'T1087.004',
  'T1088',
  'T1114',
  'T1114.003',
  'T1145'

In [51]:
net.show('graph.html')

In [5]:
techniques_by_group = {}
for grp in a.groups:
    techniques_by_group.update({
        grp: set()
    })
    
    for rel in a.relations:
        if (rel.src 
            and rel.trg
            and rel.src == grp 
            and rel.type == 'uses' 
            and rel.trg[:2] == 'T1'):
            techniques_by_group[grp].add(rel.trg)
            
groups_by_technique = {}
soft_by_technique = {}
mitigations_by_technique = {}
for tech in a.techniques:
    groups_by_technique.update({
        tech: set()
    })
    soft_by_technique.update({
        tech: set()
    })
    mitigations_by_technique.update({
        tech: set()
    })
    for rel in a.relations:
        if (rel.src 
            and rel.trg 
            and rel.trg == tech
            and rel.type == 'uses'
            and rel.src[:2] == 'G0'):
            groups_by_technique[tech].add(rel.src)
        if (rel.src 
            and rel.trg 
            and rel.trg == tech
            and rel.type == 'uses'
            and rel.src[:1] == 'S'):
            soft_by_technique[tech].add(rel.src)
        if (rel.src 
            and rel.trg 
            and rel.trg == tech
            and rel.type == 'mitigates'
            and rel.src[:1] == 'M'):
            mitigations_by_technique[tech].add(rel.src)
            
techniques_by_soft = {}
for soft in a.software:
    techniques_by_soft.update({
        soft: set()
    })
    for rel in a.relations:
        if (rel.src 
            and rel.trg
            and rel.src == soft 
            and rel.type == 'uses' 
            and rel.trg[:2] == 'T1'):
            techniques_by_soft[soft].add(rel.trg)
    
techniques_by_mitigation = {}
for mit in a.mitigations:
    techniques_by_mitigation.update({
        mit: set()
    })
    for rel in a.relations:
        if (rel.src 
            and rel.trg
            and rel.src == mit 
            and rel.type == 'mitigates' 
            and rel.trg[:2] == 'T1'):
            techniques_by_mitigation[mit].add(rel.trg)

techniques_by_data_component = {}
for dc in a.data_components:
    techniques_by_data_component.update({
        dc.name: set()
    })
    
for rel in a.relations:
    if (rel.trg 
        and rel.trg[:2] == 'T1' 
        and rel.type == 'detects'):
        for d in a.data_components:
            if d.ref == rel.source:
                techniques_by_data_component[d.name].add(rel.trg)
                
data_components_by_technique = {}
for tech in a.techniques:
    data_components_by_technique.update({
        tech: set()
    })
    for dc in techniques_by_data_component:
        if tech in techniques_by_data_component[dc]:
            data_components_by_technique[tech].add(dc)

    

ARelation(source='course-of-action--beb45abb-11e8-4aef-9778-1f9ac249784f', src='T1088', target='attack-pattern--ca1a3f50-5ebd-41f8-8320-2c7d6a6e88be', trg='T1088', description='', type='mitigates', ref='relationship--483a70b9-eae9-4d5f-925c-95c2dd7b9fa5')


In [39]:
a.collections['attack-pattern'][1]

{'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'type': 'attack-pattern',
 'name': 'ARP Cache Poisoning',
 'x_mitre_data_sources': ['Network Traffic: Network Traffic Content',
  'Network Traffic: Network Traffic Flow'],
 'x_mitre_version': '1.1',
 'modified': '2021-07-28T01:04:39.141Z',
 'created': '2020-10-15T12:05:58.755Z',
 'kill_chain_phases': [{'kill_chain_name': 'mitre-attack',
   'phase_name': 'credential-access'},
  {'kill_chain_name': 'mitre-attack', 'phase_name': 'collection'}],
 'id': 'attack-pattern--cabe189c-a0e3-4965-a473-dcff00f17213',
 'x_mitre_platforms': ['Linux', 'Windows', 'macOS'],
 'x_mitre_is_subtechnique': True,
 'x_mitre_permissions_required': ['User'],
 'description': 'Adversaries may poison Address Resolution Protocol (ARP) caches to position themselves between the communication of two or more networked devices. This activity may be used to enable follow-on behaviors such as [Network Sniffing](https://attack.mitre.org/te